Create a stack from of subtypes maps from all task and trial

In [2]:
import seaborn as sns
import os
import numpy as np
from scipy import stats, integrate
import scipy.io as sio
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

In [3]:
# load good stuff
%matplotlib inline
%load_ext rpy2.ipython
%load_ext oct2py.ipython
sns.set(color_codes=True)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#  Set path
path_root = '/home/yassinebha/Drive/HCP/subtypes_scores/26-10-2016/'
list_subtype_folder = ['subtype_5_spm_EMOTION_23-May-2017','subtype_5_spm_GAMBLING_19-Jun-2017','subtype_5_spm_LANGUAGE_18-Jun-2017',
                      'subtype_5_spm_MOTOR_19-Jun-2017','subtype_5_spm_RELATIONAL_18-Jun-2017','subtype_5_spm_SOCIAL_18-Jun-2017',
                       'subtype_5_spm_WM_18-Jun-2017']
list_subtype = ['sub1','sub2','sub3','sub4','sub5']

In [5]:
for cx,folder in enumerate(list_subtype_folder):
    path_network =  os.path.join(path_root,folder,'networks/')
    list_trial  = [f for f in os.listdir(path_network)]
    for ix,trial_name in enumerate(list_trial) :
        mat_file = os.path.join(path_network,trial_name,'subtype_{}.mat'.format(trial_name))
        %octave_push mat_file
        %octave mat_load = load(mat_file);
        %octave_pull mat_load
        subtype = mat_load.sub.map.T
        subtype_name = ['{}_{}'.format(subt,trial_name) for subt in list_subtype ]
        if ix == 0 and cx == 0:
            subtype_final = subtype
            subtype_name_final = subtype_name
        else:
            subtype_final = [np.concatenate([subtype_final[x], subtype[x]]) for x in range(np.shape(subtype)[0])]
            subtype_name_final = np.concatenate([subtype_name_final, subtype_name]) 

In [7]:
# make dataframe 
all_stack_df = pd.DataFrame(subtype_final,columns=subtype_name_final)
all_stack_df.head()

,sub1_fear,sub2_fear,sub3_fear,sub4_fear,sub5_fear,sub1_contrast_loss_event_vs_neut_event,sub2_contrast_loss_event_vs_neut_event,sub3_contrast_loss_event_vs_neut_event,sub4_contrast_loss_event_vs_neut_event,sub5_contrast_loss_event_vs_neut_event,...,sub1_places,sub2_places,sub3_places,sub4_places,sub5_places,sub1_2bk,sub2_2bk,sub3_2bk,sub4_2bk,sub5_2bk
0,-0.000026,-0.000010,3.400174e-06,1.969976e-05,0.000001,-2.722667e-04,0.000034,3.307610e-05,0.000092,-0.000035,...,7.089784e-05,0.000071,2.997301e-06,-0.000042,-0.000037,0.000049,-0.000026,0.000066,-0.000129,3.228139e-06
1,-0.000030,-0.000050,7.557251e-05,1.935898e-05,-0.000020,-2.416968e-05,-0.000024,-9.189761e-05,0.000128,-0.000043,...,5.864750e-05,-0.000017,3.228650e-05,-0.000051,-0.000006,0.000039,-0.000018,0.000081,-0.000160,8.092133e-06
2,-0.000037,-0.000062,2.097875e-04,-5.504649e-05,-0.000034,7.750296e-05,0.000105,-1.515700e-04,0.000077,-0.000081,...,3.866714e-05,0.000037,-2.390819e-06,0.000006,-0.000032,-0.000008,-0.000013,0.000030,-0.000075,3.030099e-05
3,0.000001,0.000018,-1.056633e-05,2.286825e-06,-0.000005,-8.490994e-06,0.000001,6.710994e-06,0.000010,-0.000013,...,-2.469192e-07,0.000003,-6.788072e-07,-0.000007,0.000003,0.000010,-0.000011,-0.000003,0.000002,2.005255e-06
4,-0.000007,0.000009,5.834449e-07,7.194440e-07,-0.000002,8.046937e-07,0.000002,5.579041e-07,-0.000009,0.000007,...,9.066289e-07,0.000009,-9.413969e-07,-0.000012,0.000003,0.000006,-0.000010,-0.000004,0.000011,4.487713e-07


In [8]:
# Make matrix out of the remaining guys except subject ID column
stack_mat = all_stack_df.as_matrix()
# Normalize these guys by column
stack_norm = (stack_mat - np.mean(stack_mat, 0))/np.std(stack_mat, 0)

In [10]:
# Save normalized pheno to be used in octave for bootsrap analysis
ind = all_stack_df.index
stack_norm_df = pd.DataFrame(stack_norm, columns=all_stack_df.columns.get_values(),index=ind)

In [11]:
stack_norm_df.head()

,sub1_fear,sub2_fear,sub3_fear,sub4_fear,sub5_fear,sub1_contrast_loss_event_vs_neut_event,sub2_contrast_loss_event_vs_neut_event,sub3_contrast_loss_event_vs_neut_event,sub4_contrast_loss_event_vs_neut_event,sub5_contrast_loss_event_vs_neut_event,...,sub1_places,sub2_places,sub3_places,sub4_places,sub5_places,sub1_2bk,sub2_2bk,sub3_2bk,sub4_2bk,sub5_2bk
0,0.445295,-0.164905,-0.322196,0.723914,-0.480451,-1.425723,1.101184,0.113329,-0.738411,0.674656,...,0.512828,-0.204082,0.369714,-0.648153,-0.105009,0.413560,0.546331,0.676559,-0.442354,-0.610221
1,0.434186,-0.314472,-0.155334,0.722393,-0.573545,-0.879564,0.932399,-0.418848,-0.617315,0.649279,...,0.480156,-0.342678,0.483918,-0.696465,0.308871,0.391364,0.626150,0.737690,-0.485098,-0.586861
2,0.416064,-0.359790,0.154968,0.390277,-0.634017,-0.655743,1.302711,-0.672952,-0.785976,0.524565,...,0.426868,-0.256480,0.348704,-0.387892,-0.038697,0.284835,0.674181,0.536919,-0.368107,-0.480200
3,0.513731,-0.058470,-0.354486,0.646190,-0.506626,-0.845049,1.006189,0.001058,-1.009320,0.747586,...,0.323083,-0.310923,0.355380,-0.459530,0.429180,0.325493,0.691129,0.407454,-0.260357,-0.616094
4,0.493780,-0.092133,-0.328708,0.639193,-0.493909,-0.824586,1.007507,-0.025143,-1.070359,0.813138,...,0.326160,-0.301969,0.354356,-0.484791,0.424466,0.316114,0.699989,0.405333,-0.248753,-0.623570


In [13]:
# save final stack
stack_norm_df.to_csv(os.path.join(path_root,'pheno/all_subtype_stack.csv'),index=False)